#  Breast cancer detection

## Učitavanje biblioteka

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import time

In [ ]:
#funkcija za računanje metrika
def metrike(modeli, testData, y_test, time):
    for ind, model in enumerate(modeli):
        print(F"Algoritam: {type(model.optimizer).__name__}")
        #ako je ANN
        y_pred = model.predict(testData)
        y_pred = (y_pred > 0.5)
        print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
        print('ROC AUC:', metrics.roc_auc_score(y_test, y_pred))
        print("Confusion matrix")
        CM = metrics.confusion_matrix(y_test, y_pred)
        TN = CM[0, 0]
        TP = CM[1, 1]
        FP = CM[0, 1]
        FN = CM[1, 0]
        print("    P0           P1")
        print(f"S0  {TN}        {FP}")
        print(f"S1  {FN}        {TP}")
        print("Recall: ",metrics.recall_score(y_test,y_pred))
        print("Precision: ",  metrics.precision_score(y_test, y_pred))
        print("F1 score: ", metrics.f1_score(y_test, y_pred))
        print(f"Training time (sec): {time[ind]}" )
        print("-----------------------------------------------")
def acc_loss_plot(histories, model, ind=0):
        plt.plot(pd.DataFrame(histories[ind].history["accuracy"]))
        plt.plot(pd.DataFrame(histories[ind].history["val_accuracy"]))
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title(f'{type(model.optimizer).__name__} accuracy')
        plt.figure(figsize=(6,6), dpi=500)
        plt.show()

        plt.plot(pd.DataFrame(histories[ind].history["loss"]))
        plt.plot(pd.DataFrame(histories[ind].history["val_loss"]))
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'{type(model.optimizer).__name__} loss')
        plt.figure(figsize=(6,6), dpi=500)
        plt.show()

def metrike_table(modeli, testData, y_test, tr_time, histories):
    data = []
    for ind, model in enumerate(modeli):
        start = time.time()
        y_pred = model.predict(testData)
        y_pred = (y_pred > 0.5)
        test_time = time.time()-start
        data.append([
            type(model.optimizer).__name__,
            metrics.accuracy_score(y_test, y_pred),
            metrics.roc_auc_score(y_test, y_pred),
            metrics.recall_score(y_test,y_pred),
            metrics.precision_score(y_test, y_pred),
            metrics.f1_score(y_test, y_pred),
            tr_time[ind],
            test_time,
            len(histories[ind].history['loss'])
            ])
        acc_loss_plot(histories, model, ind)
    df = pd.DataFrame(data, columns = ['Algoritam', "Accuracy",'ROC AUC',"Recall","Precision","F1 score", "Training time (sec)","Test time (sec)",'Epochs'])
    return df

## Učitavanje skupa podataka

In [ ]:
dataset = pd.read_csv('../datasets/breast_cancer_wisconsin.csv')
dataset.head()

## Analiza skupa podataka

In [ ]:
dataset.dtypes
#sve numeričke varijable

In [ ]:
dataset.describe()

In [ ]:
dataset['diagnosis'].value_counts()

In [ ]:
benign = dataset['diagnosis'].value_counts()['B']
malign = dataset['diagnosis'].value_counts()['M']

In [ ]:
plt.figure(figsize=(5,5))
plt.bar(['Benign','Malign'],[benign,malign])
plt.ylabel("Number of occurrences")
plt.title("Benign/Malign")
plt.show()

In [ ]:
dataset['diagnosis'].value_counts()['B']/len(dataset)*100
#62.74% instanci klase Benign

In [ ]:
dataset['diagnosis'] = dataset['diagnosis'].replace(['B','M'],[0,1])

In [ ]:
dataset['diagnosis'].value_counts()

In [ ]:
pd.DataFrame(dataset.corr()['diagnosis'].drop('diagnosis'),dataset.drop(['diagnosis'], axis=1).columns)

In [ ]:
korelacijska_matrica = dataset.corr()
plt.subplots(figsize=(10,10))
sns.heatmap(korelacijska_matrica, vmax=0.9, square=True)

## Čišćenje

In [ ]:
dataset["id"].is_unique

In [ ]:
dataset = dataset.drop(['id'],axis=1)

In [ ]:
#nema null vrijednosti
dataset.isna().sum()

In [ ]:
X=dataset.drop(['diagnosis'], axis=1)
Y=dataset['diagnosis']

## Podjela na trening i test skupove

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state=42)

In [ ]:
scX = StandardScaler()
X_train = scX.fit_transform(X_train)
X_test = scX.transform(X_test)

In [ ]:
print(X_train.shape,y_train.shape)

## ANN - ADAM

In [ ]:
ann_adam = tf.keras.models.Sequential()
#3 skirvena sloja sa po 8, 16 i 32 neurona
ann_adam.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann_adam.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann_adam.add(tf.keras.layers.Dense(units=32, activation='relu'))
# Dodavanje izlaznog sloja
ann_adam.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# kompajliranje mreže
ann_adam.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(f"R2 score:{metrics.r2_score(y_test,grid_result.predict(X_test))}")

In [ ]:
start = time.time()
adam_history = ann_adam.fit(X_train, y_train, batch_size = 16, epochs = 10, validation_data=(X_test,y_test))
adam_time = time.time()-start

In [ ]:
metrike([ann_adam], X_test, y_test, [adam_time])

In [ ]:
acc_loss_plot([adam_history], ann_adam, 0)

## ANN - ADAMAX

In [ ]:
ann_adamax = tf.keras.models.Sequential()
ann_adamax.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann_adamax.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann_adamax.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann_adamax.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann_adamax.compile(optimizer = 'adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
start = time.time()
adamax_history = ann_adamax.fit(X_train, y_train, batch_size = 16, epochs = 50, validation_data=(X_test,y_test))
adamax_time = time.time()-start

In [ ]:
metrike([ann_adamax], X_test, y_test, [adamax_time])

In [ ]:
acc_loss_plot([adamax_history], ann_adamax, 0)

## ANN - ADAGRAD

In [ ]:

ann_adagrad = tf.keras.models.Sequential()
ann_adagrad.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann_adagrad.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann_adagrad.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann_adagrad.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann_adagrad.compile(optimizer = 'adagrad', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
start = time.time()
adagrad_history = ann_adagrad.fit(X_train, y_train, batch_size = 16, epochs = 1000, validation_data=(X_test, y_test))
adagrad_time = time.time()-start

In [ ]:
metrike([ann_adagrad], X_test, y_test, [adagrad_time])

In [ ]:
acc_loss_plot([adagrad_history], ann_adagrad, 0)

## ANN - NADAM

In [ ]:

ann_nadam = tf.keras.models.Sequential()
ann_nadam.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann_nadam.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann_nadam.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann_nadam.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann_nadam.compile(optimizer = 'nadam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
start = time.time()
nadam_history = ann_nadam.fit(X_train, y_train, batch_size = 16, epochs = 12, validation_data=(X_test,y_test))
nadam_time = time.time()-start

In [ ]:
metrike([ann_nadam], X_test, y_test, [nadam_time])

In [ ]:
acc_loss_plot([nadam_history], ann_nadam, 0)

## ANN - SGD

In [ ]:
ann_sgd = tf.keras.models.Sequential()
ann_sgd.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann_sgd.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann_sgd.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann_sgd.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann_sgd.compile(optimizer = 'SGD', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
start = time.time()
sgd_history = ann_sgd.fit(X_train, y_train, batch_size = 16, epochs = 70, validation_data=(X_test,y_test))
sgd_time = time.time()-start

In [ ]:
metrike([ann_sgd], X_test, y_test, [sgd_time])

In [ ]:
acc_loss_plot([sgd_history], ann_sgd, 0)

## ANN - RMSprop

In [ ]:
ann_rms = tf.keras.models.Sequential()
ann_rms.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann_rms.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann_rms.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann_rms.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann_rms.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
start = time.time()
rms_history = ann_rms.fit(X_train, y_train, batch_size = 16, epochs = 10, validation_data=(X_test,y_test))
rms_time = time.time()-start

In [ ]:
metrike([ann_rms], X_test, y_test, [rms_time])

In [ ]:
acc_loss_plot([rms_history], ann_rms, 0)

In [ ]:
models = [ann_adam,ann_adamax,ann_adagrad,ann_nadam,ann_sgd,ann_rms]
times = [adam_time,adamax_time,adagrad_time, nadam_time, sgd_time,rms_time]
histories = [adam_history,adamax_history,adagrad_history,nadam_history,sgd_history,rms_history]
metrike_table(models, X_test, y_test, times, histories)

In [ ]:
for model in models:
    model.save(f'../saved_models/breast_cancer/{type(model.optimizer).__name__}')